**CARGA DE LIBRERÍAS**

In [2]:
!pip install pandas matplotlib seaborn reportlab openpyxl python-docx docx2pdf
!apt-get update && apt-get install -y libreoffice

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.1 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,282 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/

**GENERACIÓN DE INFORME**

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import textwrap
from datetime import datetime
import locale
import subprocess # Para ejecutar LibreOffice

# --- IMPORTACIONES PARA DOCX ---
from docx import Document
from docx.shared import Inches, Pt, RGBColor
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.oxml import OxmlElement
from docx.oxml.ns import qn
from matplotlib.ticker import FixedLocator

import matplotlib
matplotlib.use('Agg')

# --- CONFIGURACIÓN DE LOCALE ---
try:
    locale.setlocale(locale.LC_TIME, 'es_CO.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
    except locale.Error:
        try:
            locale.setlocale(locale.LC_TIME, '')
            print("Advertencia: Locale 'es_CO.UTF-8' o 'es_ES.UTF-8' no encontrado. Usando locale por defecto.")
        except locale.Error:
            print("Advertencia: No se pudo configurar el locale para fechas en español.")

# --- CONFIGURACIÓN DE GRÁFICOS ---
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial', 'Helvetica', 'DejaVu Sans']
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['xtick.labelsize'] = 9.0 # Mantenido en 9.0
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 9.5
plt.rcParams['legend.title_fontsize'] = 10.5
plt.rcParams['figure.titlesize'] = 14
sns.set_style("ticks")

# --- RUTAS DE ARCHIVOS (Actualizadas) ---
try:
    import google.colab
    ruta_base = "/content/drive/MyDrive/EVALUACIÓN DOCENTE/"
    print("Entorno de Google Colab detectado (solo informativo).")
except ImportError:
    ruta_base = "./"
    print(f"Entorno local detectado. Ruta base: '{ruta_base}'")

ruta_datos_plantilla = os.path.join(ruta_base, "DATOS-PLANTILLA")
ruta_informe_salida = os.path.join(ruta_base, "INFORME")
ruta_graficas_salida = os.path.join(ruta_base, "graficas_docente_espacio_docx")

ruta_excel = os.path.join(ruta_datos_plantilla, "Seguimiento a Syllabus(5001-6425).xlsx")
ruta_plantilla_docx = os.path.join(ruta_datos_plantilla, "plantilla.docx")
carpeta_graficas = ruta_graficas_salida
# Actualizar nombres de archivo para reflejar la versión
temp_docx_output_path = os.path.join(ruta_informe_salida, "Informe_Evaluacion_Syllabus.docx")
final_pdf_output_path = os.path.join(ruta_informe_salida, "Informe_Evaluacion_Syllabus.pdf")

os.makedirs(ruta_informe_salida, exist_ok=True)
os.makedirs(carpeta_graficas, exist_ok=True)

# --- DATOS GLOBALES PARA TOC ---
toc_entries = []
_global_bookmark_id_counter = 0

# --- FUNCIONES AUXILIARES PARA MARCAdORES E HIPERVÍNCULOS ---
def generate_bookmark_name(text, prefix="bm_"):
    valid_chars = "".join(c if c.isalnum() else '_' for c in text)
    valid_chars = '_'.join(filter(None, valid_chars.split('_')))
    bookmark = prefix + valid_chars
    return bookmark[:39]

def add_bookmark(paragraph, bookmark_name):
    global _global_bookmark_id_counter
    _global_bookmark_id_counter += 1
    bookmark_id = str(_global_bookmark_id_counter)

    bm_start_el = OxmlElement('w:bookmarkStart')
    bm_start_el.set(qn('w:id'), bookmark_id)
    bm_start_el.set(qn('w:name'), bookmark_name)
    paragraph._p.insert(0, bm_start_el)

    bm_end_el = OxmlElement('w:bookmarkEnd')
    bm_end_el.set(qn('w:id'), bookmark_id)
    paragraph._p.append(bm_end_el)

def add_hyperlink_to_bookmark(doc_paragraph, display_text, bookmark_name,
                              font_name='Arial', font_size=11, color_hex='000000',
                              is_italic_for_toc=False, is_bold_for_toc=False, toc_level=0):
    hyperlink = OxmlElement('w:hyperlink')
    hyperlink.set(qn('w:anchor'), bookmark_name)

    sub_run_el = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')

    run_font = OxmlElement('w:rFonts')
    if font_name:
        run_font.set(qn('w:ascii'), font_name); run_font.set(qn('w:hAnsi'), font_name)
        run_font.set(qn('w:eastAsia'), font_name); run_font.set(qn('w:cs'), font_name)
    rPr.append(run_font)

    run_size = OxmlElement('w:sz'); run_size.set(qn('w:val'), str(font_size * 2)); rPr.append(run_size)
    run_size_cs = OxmlElement('w:szCs'); run_size_cs.set(qn('w:val'), str(font_size * 2)); rPr.append(run_size_cs)

    if color_hex:
        run_color = OxmlElement('w:color'); run_color.set(qn('w:val'), color_hex.replace("#","")); rPr.append(run_color)

    if is_italic_for_toc:
        italic_el = OxmlElement('w:i'); rPr.append(italic_el)

    if is_bold_for_toc:
        bold_el = OxmlElement('w:b'); rPr.append(bold_el)

    sub_run_el.append(rPr)

    text_el = OxmlElement('w:t'); text_el.text = display_text; sub_run_el.append(text_el)
    hyperlink.append(sub_run_el)

    doc_paragraph._element.append(hyperlink)

    if toc_level > 0:
        doc_paragraph.paragraph_format.left_indent = Inches(0.25 * toc_level)
    doc_paragraph.paragraph_format.space_after = Pt(3)

# --- CARGA DE DATOS ---
try:
    df_original = pd.read_excel(ruta_excel)
except FileNotFoundError:
    print(f"Error: Archivo Excel no encontrado en: {ruta_excel}")
    print("Asegúrate de que la carpeta 'DATOS-PLANTILLA' exista en la ruta base y contenga el archivo.")
    exit()
except Exception as e:
    print(f"Error al leer Excel: {e}"); exit()

df = df_original[df_original['Corte Academico'] == 'Primer Corte 2025 - 1'].copy()
if df.empty:
    print("DataFrame vacío tras filtrar. No se generará informe."); exit()

preguntas = [
    "Las temáticas desarrolladas corresponden a las registradas en el syllabus.",
    "Las temáticas o contenidos se desarrollan acorde a los tiempos planeados en el syllabus",
    "Las actividades didácticas desarrolladas son acordes a la temática que se está abordado",
    "La estrategia didáctica es pertinente para el desarrollo de los resultados de aprendizaje que se pretende desarrollar.",
    "Los mecanismos (estrategias) de evaluación son pertinentes para comprobar los resultados de aprendizaje que se esperan desarrollar en el espacio académico.",
    "Se implementan diferentes metodologías de evaluación"
]
orden_etiquetas = ['Siempre', 'Muy a menudo', 'A menudo', 'Raras veces', 'Casi Nunca']
colores_palette = ["#1a73e8", "#4285f4", "#fbbc04", "#ea4335", "#c53929"]
paleta_colores = dict(zip(orden_etiquetas, colores_palette))
docentes = df['Docente del Espacio Académico'].dropna().unique()

total_respuestas_global = 0
if preguntas and not df.empty and preguntas[0] in df.columns:
    total_respuestas_global = df[preguntas[0]].count()
    print(f"Total de respuestas global calculado: {total_respuestas_global}")
else:
    print("Advertencia: No se pudo calcular el total de respuestas global.")


# --- FUNCIONES DE ESTILO Y CONTENIDO ---
def add_styled_paragraph(doc, text_content, style_name=None, font_name='Arial', font_size=11, is_bold=False, is_italic=False, align=WD_ALIGN_PARAGRAPH.LEFT, space_before_pt=0, space_after_pt=6, color_hex=None,
                         is_heading_for_toc=False, toc_display_text=None, toc_level=0,
                         collect_toc_entry_if_heading=False):

    if style_name and style_name in [s.name for s in doc.styles if s.type == WD_STYLE_TYPE.PARAGRAPH]:
        p = doc.add_paragraph(text_content, style=style_name)
    else:
        p = doc.add_paragraph()
        run = p.add_run(text_content)
        if font_name:
            run.font.name = font_name
        run.font.size = Pt(font_size)
        run.bold = is_bold
        run.italic = is_italic
        if color_hex:
            try: run.font.color.rgb = RGBColor.from_string(color_hex.replace("#",""))
            except ValueError: print(f"Advertencia: Color '{color_hex}' no válido.")

    p.alignment = align
    p.paragraph_format.space_before = Pt(space_before_pt)
    p.paragraph_format.space_after = Pt(space_after_pt)

    if is_heading_for_toc:
        actual_toc_display_text = toc_display_text if toc_display_text is not None else text_content
        bookmark_name = generate_bookmark_name(text_content)
        add_bookmark(p, bookmark_name)

        if collect_toc_entry_if_heading:
             if not any(entry['bookmark'] == bookmark_name for entry in toc_entries):
                 toc_entries.append({'text': actual_toc_display_text, 'bookmark': bookmark_name, 'level': toc_level})
    return p

def build_portada(doc):
    # ... (código de build_portada sin cambios) ...
    corte_academico_valor = df['Corte Academico'].unique()[0] if not df.empty and 'Corte Academico' in df.columns and len(df['Corte Academico'].unique()) > 0 else "No especificado"
    fecha_min_str, fecha_max_str = "No disponible", "No disponible"
    fmt_date = '%d de %B de %Y'; fallback_fmt_date = '%Y-%m-%d'
    if 'Hora de inicio' in df.columns and not df.empty:
        df_temp_fechas = df.copy(); df_temp_fechas['Hora de inicio'] = pd.to_datetime(df_temp_fechas['Hora de inicio'], errors='coerce')
        fechas_validas = df_temp_fechas['Hora de inicio'].dropna()
        if not fechas_validas.empty:
            try: fecha_min_str = fechas_validas.min().strftime(fmt_date)
            except ValueError: fecha_min_str = fechas_validas.min().strftime(fallback_fmt_date)
            try: fecha_max_str = fechas_validas.max().strftime(fmt_date)
            except ValueError: fecha_max_str = fechas_validas.max().strftime(fallback_fmt_date)
    try: fecha_generacion_informe_str = datetime.now().strftime(fmt_date)
    except ValueError: fecha_generacion_informe_str = datetime.now().strftime(fallback_fmt_date)
    add_styled_paragraph(doc, "", space_after_pt=50)
    add_styled_paragraph(doc, "INFORME DE EVALUACIÓN DE SYLLABUS", font_name='Arial', font_size=22, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=12, space_after_pt=20, color_hex='003366')
    add_styled_paragraph(doc, "", space_after_pt=30)
    add_styled_paragraph(doc, "Facultad de Ingeniería Industrial", font_name='Arial', font_size=16, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=5, color_hex='202124')
    add_styled_paragraph(doc, "Seccional Villavicencio", font_name='Arial', font_size=16, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=5, color_hex='202124')
    add_styled_paragraph(doc, "Universidad Santo Tomás", font_name='Arial', font_size=16, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=5, color_hex='202124')
    add_styled_paragraph(doc, "", space_after_pt=40)
    lines_info1 = f"Periodo Analizado: {corte_academico_valor}".split('<br/>')
    for i, line in enumerate(lines_info1):
        add_styled_paragraph(doc, line, font_name='Arial', font_size=12, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=10 if i==0 else 0 , space_after_pt=(10 if i == len(lines_info1) - 1 else 0), color_hex='2F4F4F')
    lines_info2 = f"La información corresponde a la recopilación de datos:<br/>Desde el {fecha_min_str} hasta el {fecha_max_str}".split('<br/>')
    for i, line in enumerate(lines_info2):
        add_styled_paragraph(doc, line, font_name='Arial', font_size=12, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=10 if i==0 else 0 , space_after_pt=(10 if i == len(lines_info2) - 1 else 0), color_hex='2F4F4F')
    add_styled_paragraph(doc, "", space_after_pt=40)
    add_styled_paragraph(doc, "Elaborado por:", font_name='Arial', font_size=12, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=20, space_after_pt=4, color_hex='2F4F4F')
    add_styled_paragraph(doc, "Adriana Amelia Cespedes Orjuela", font_name='Arial', font_size=12, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=2, color_hex='2F4F4F')
    add_styled_paragraph(doc, "Líder de Currículo de la Facultad", font_name='Arial', font_size=11, is_italic=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_after_pt=15, color_hex='2F4F4F')
    add_styled_paragraph(doc, "", space_after_pt=30)
    add_styled_paragraph(doc, f"Fecha de generación del informe: {fecha_generacion_informe_str}", font_name='Arial', font_size=9, is_italic=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=5, space_after_pt=4, color_hex='808080')
    add_styled_paragraph(doc, "Generado en Villavicencio, Meta, Colombia.", font_name='Arial', font_size=9, is_italic=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=5, space_after_pt=4, color_hex='808080')

def build_table_of_contents(doc, current_toc_entries):
    # ... (código de build_table_of_contents sin cambios) ...
    add_styled_paragraph(doc, "Tabla de Contenido", font_name='Arial', font_size=18, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=12, space_after_pt=20)
    if not current_toc_entries:
        add_styled_paragraph(doc, "No hay entradas para la tabla de contenido.", space_after_pt=2)
    else:
        for entry in current_toc_entries:
            p_toc_entry = doc.add_paragraph()
            is_main_title = (entry['level'] == 0)
            display_text_for_toc = f"•   {entry['text']}" if is_main_title else entry['text']
            add_hyperlink_to_bookmark(p_toc_entry, display_text_for_toc, entry['bookmark'],
                                      font_size=11, toc_level=entry['level'],
                                      is_italic_for_toc=(entry['level'] > 0),
                                      is_bold_for_toc=is_main_title,
                                      color_hex='000000')

def build_introductory_section(doc, num_total_respuestas, collect_toc_entry_mode=True):
    """Construye la sección introductoria."""
    title_intro = "1. Informe de Seguimiento a Syllabus"
    add_styled_paragraph(doc, title_intro, font_name='Arial', font_size=14, is_bold=True, align=WD_ALIGN_PARAGRAPH.LEFT, space_before_pt=12, space_after_pt=10, is_heading_for_toc=True, toc_level=0, collect_toc_entry_if_heading=collect_toc_entry_mode, toc_display_text=title_intro)

    add_styled_paragraph(doc, "El seguimiento a syllabus tiene como objetivo revisar la articulación entre sus diferentes elementos y su contribución al logro de los resultados de aprendizaje y competencias.", font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=8)
    add_styled_paragraph(doc, "Es por esto, que desde la facultad de Ingeniería Industrial dando seguimiento a los lineamientos dispuestos en el protocolo de seguimiento a syllabus, documento DO-N-IN- 001, registrado en el sistema de gestión de calidad, pone en marcha el procedimiento de seguimiento a syllabus a través de una encuesta que será compartida a través de One drive.", font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=10)

    add_styled_paragraph(doc, "1.1. Instrumento", font_name='Arial', font_size=12, is_bold=True, align=WD_ALIGN_PARAGRAPH.LEFT, space_before_pt=10, space_after_pt=6)
    add_styled_paragraph(doc, "El instrumento que se puso en marcha en la Facultad de Ingeniería Industrial es una encuesta de percepción que contiene las siguientes preguntas:", font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=6)

    preguntas_instrumento = [
        "Las temáticas desarrolladas corresponden a las registradas en el syllabus.",
        "Las temáticas o contenidos se desarrollan acorde a los tiempos planeados en el syllabus.",
        "Las actividades didácticas desarrolladas son acordes a la temática que se está abordado.",
        "La estrategia didáctica es pertinente para el desarrollo de los resultados de aprendizaje que se pretende desarrollar.",
        "Los mecanismos (estrategias) de evaluación son pertinentes para comprobar los resultados de aprendizaje que se esperan desarrollar en el espacio académico.",
        "Se implementan diferentes metodologías de evaluación."
    ]
    for pregunta_inst in preguntas_instrumento:
        p_inst = add_styled_paragraph(doc, f"•   {pregunta_inst}", font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=3)
        p_inst.paragraph_format.left_indent = Inches(0.25)

    add_styled_paragraph(doc, "Preguntas que contarán con las siguientes opciones de respuesta:", font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_before_pt=8, space_after_pt=6)
    opciones_respuesta_intro = ["Siempre", "Muy a menudo", "A menudo", "Raras veces", "Casi Nunca"]
    for opcion_resp in opciones_respuesta_intro:
        p_opc = add_styled_paragraph(doc, f"•   {opcion_resp}", font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=3)
        p_opc.paragraph_format.left_indent = Inches(0.25)

    add_styled_paragraph(doc, "1.2. Evaluación", font_name='Arial', font_size=12, is_bold=True, align=WD_ALIGN_PARAGRAPH.LEFT, space_before_pt=10, space_after_pt=6)
    texto_evaluacion = f"La aplicación de la encuesta, se lleva a cabo a través de la herramienta de formularios de Microsoft 365, en la cual, un total de {num_total_respuestas} estudiantes dieron respuesta a la herramienta de evaluación. Distribuidos en cada uno de los espacios académicos del programa, evalúan el desempeño de los docentes, por espacio académico."
    add_styled_paragraph(doc, texto_evaluacion, font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=8)
    add_styled_paragraph(doc, "A continuación, se presentan los resultados desglosados por docente de manera particular.", font_size=11, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=12)


def build_main_content_graphics(doc, collect_toc_entry_mode=True):
    """Construye las secciones con gráficas."""
    # --- 2. EVALUACIÓN GENERAL DE SYLLABUS EN LA FACULTAD (Renumerado) ---
    title1 = "2. EVALUACIÓN GENERAL DE SYLLABUS EN LA FACULTAD"
    add_styled_paragraph(doc, title1, font_name='Arial', font_size=16, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=12, space_after_pt=12, color_hex='003366', is_heading_for_toc=True, toc_level=0, collect_toc_entry_if_heading=collect_toc_entry_mode, toc_display_text=title1)
    # ... (resto del código de la sección 1/gráficas generales) ...
    add_styled_paragraph(doc, "", space_after_pt=10)
    total_respuestas_dataset_seccion1 = df[preguntas[0]].count() if preguntas and not df.empty and preguntas[0] in df.columns else 0
    add_styled_paragraph(doc, f"Total de respuestas en el corte: {total_respuestas_dataset_seccion1}", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=10)
    datos_generales_plot = []
    if preguntas and not df.empty:
        for pregunta_texto_completo in preguntas:
            if pregunta_texto_completo not in df.columns: continue
            df_pregunta_valida = df[df[pregunta_texto_completo].notna()]
            total_respuestas_pregunta_general = len(df_pregunta_valida)
            if total_respuestas_pregunta_general > 0:
                conteo_general = df_pregunta_valida[pregunta_texto_completo].value_counts().reindex(orden_etiquetas, fill_value=0)
                porcentajes_general = (conteo_general / total_respuestas_pregunta_general) * 100
                for respuesta, porcentaje in porcentajes_general.items():
                    datos_generales_plot.append({'Pregunta_Grafico': pregunta_texto_completo, 'Respuesta': respuesta, 'Porcentaje': porcentaje})
    if datos_generales_plot:
        df_plot_general = pd.DataFrame(datos_generales_plot)
        df_plot_general['Pregunta_Grafico'] = pd.Categorical(df_plot_general['Pregunta_Grafico'], categories=preguntas, ordered=True)
        df_plot_general = df_plot_general.sort_values('Pregunta_Grafico')
        fig_gen, ax_gen = plt.subplots(figsize=(11, 7)); fig_gen.set_facecolor('white'); ax_gen.set_facecolor('#f8f9fa')
        sns.barplot(data=df_plot_general, x='Pregunta_Grafico', y='Porcentaje', hue='Respuesta', palette=paleta_colores, hue_order=orden_etiquetas, ax=ax_gen, width=0.8, saturation=0.85)
        ax_gen.set_xlabel(''); ax_gen.set_ylabel('Porcentaje (%)', fontsize=10, labelpad=10)
        ax_gen.spines['top'].set_visible(False); ax_gen.spines['right'].set_visible(False); ax_gen.spines['left'].set_linewidth(0.5); ax_gen.spines['bottom'].set_linewidth(0.5)
        ax_gen.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd'); ax_gen.set_yticks(range(0, 101, 20)); ax_gen.set_yticklabels([f'{x}%' for x in range(0, 101, 20)], fontsize=9)
        handles, labels = ax_gen.get_legend_handles_labels()
        if ax_gen.get_legend() is not None: ax_gen.get_legend().remove()
        legend_gen = fig_gen.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.97), ncol=len(orden_etiquetas), title='Respuesta', fontsize=9.5, frameon=False)
        plt.setp(legend_gen.get_title(), fontweight='bold', fontsize=10.5)
        tick_labels = [textwrap.fill(pregunta, width=20, break_long_words=False, replace_whitespace=False) for pregunta in preguntas]
        ax_gen.set_xticks(range(len(preguntas)))
        ax_gen.set_xticklabels(tick_labels, rotation=0, ha="center", fontsize=9.0) # Tamaño de fuente aumentado
        plt.subplots_adjust(top=0.88, bottom=0.25, left=0.08, right=0.95)
        ruta_imagen_general = os.path.join(carpeta_graficas, "grafico_distribucion_general.png")
        plt.savefig(ruta_imagen_general, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_gen)
        try: doc.add_picture(ruta_imagen_general, width=Inches(6.5))
        except Exception as e: print(f"Error al añadir imagen general: {e}")
        add_styled_paragraph(doc, "", space_after_pt=15)
    else: add_styled_paragraph(doc, "No hay datos suficientes para gráfico general.", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY)
    doc.add_page_break()

    # --- 3. ANÁLISIS GENERAL DE SYLLABUS POR DOCENTE (Renumerado) ---
    title2 = "3. ANÁLISIS GENERAL DE SYLLABUS POR DOCENTE"
    add_styled_paragraph(doc, title2, font_name='Arial', font_size=16, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=12, space_after_pt=12, color_hex='003366', is_heading_for_toc=True, toc_level=0, collect_toc_entry_if_heading=collect_toc_entry_mode, toc_display_text=title2)
    # ... (resto del código de la sección 2/gráficas por docente) ...
    add_styled_paragraph(doc, "", space_after_pt=10)
    for docente_agg_idx, docente_agg in enumerate(docentes):
        docente_title_full = f"Docente: {docente_agg}"
        add_styled_paragraph(doc, docente_title_full, font_name='Arial', font_size=14, is_bold=True, space_before_pt=10, space_after_pt=8, color_hex='1a73e8', is_heading_for_toc=True, toc_display_text=docente_agg, toc_level=1, collect_toc_entry_if_heading=collect_toc_entry_mode)
        add_styled_paragraph(doc, "", space_after_pt=6)
        df_docente_agg = df[df['Docente del Espacio Académico'] == docente_agg]
        if df_docente_agg.empty:
            add_styled_paragraph(doc, "No hay datos para este docente.", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY)
            if docente_agg_idx < len(docentes) -1 : doc.add_page_break()
            continue
        datos_docente_agg_plot = []
        n_total_respuestas_docente_agg = df_docente_agg[preguntas[0]].count() if preguntas and preguntas[0] in df_docente_agg.columns else 0
        # ... (lógica de preparación de datos para gráfico de docente) ...
        for pregunta_texto_completo in preguntas:
            if pregunta_texto_completo not in df_docente_agg.columns: continue
            df_docente_pregunta_valida = df_docente_agg[df_docente_agg[pregunta_texto_completo].notna()]
            total_respuestas_docente_pregunta = len(df_docente_pregunta_valida)
            if total_respuestas_docente_pregunta > 0:
                conteo_docente_agg = df_docente_pregunta_valida[pregunta_texto_completo].value_counts().reindex(orden_etiquetas, fill_value=0)
                porcentajes_docente_agg = (conteo_docente_agg / total_respuestas_docente_pregunta) * 100
                for respuesta, porcentaje in porcentajes_docente_agg.items():
                    datos_docente_agg_plot.append({'Pregunta_Grafico': pregunta_texto_completo, 'Respuesta': respuesta, 'Porcentaje': porcentaje})
        if datos_docente_agg_plot:
            # ... (código de generación y guardado del gráfico de docente) ...
            df_plot_docente_agg = pd.DataFrame(datos_docente_agg_plot)
            df_plot_docente_agg['Pregunta_Grafico'] = pd.Categorical(df_plot_docente_agg['Pregunta_Grafico'], categories=preguntas, ordered=True)
            df_plot_docente_agg = df_plot_docente_agg.sort_values('Pregunta_Grafico')
            fig_doc_agg, ax_doc_agg = plt.subplots(figsize=(11, 7)); fig_doc_agg.set_facecolor('white'); ax_doc_agg.set_facecolor('#f8f9fa')
            sns.barplot(data=df_plot_docente_agg, x='Pregunta_Grafico', y='Porcentaje', hue='Respuesta', palette=paleta_colores, hue_order=orden_etiquetas, ax=ax_doc_agg, width=0.8, saturation=0.85)
            ax_doc_agg.set_xlabel(''); ax_doc_agg.set_ylabel('Porcentaje (%)', fontsize=10, labelpad=10)
            ax_doc_agg.spines['top'].set_visible(False); ax_doc_agg.spines['right'].set_visible(False); ax_doc_agg.spines['left'].set_linewidth(0.5); ax_doc_agg.spines['bottom'].set_linewidth(0.5)
            ax_doc_agg.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd'); ax_doc_agg.set_yticks(range(0, 101, 20)); ax_doc_agg.set_yticklabels([f'{x}%' for x in range(0, 101, 20)], fontsize=9)
            handles, labels = ax_doc_agg.get_legend_handles_labels()
            if ax_doc_agg.get_legend() is not None: ax_doc_agg.get_legend().remove()
            legend_doc_agg = fig_doc_agg.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.97), ncol=len(orden_etiquetas), title='Respuesta', fontsize=9.5, frameon=False)
            plt.setp(legend_doc_agg.get_title(), fontweight='bold', fontsize=10.5)
            tick_labels_doc = [textwrap.fill(pregunta, width=20, break_long_words=False, replace_whitespace=False) for pregunta in preguntas]
            ax_doc_agg.set_xticks(range(len(preguntas)))
            ax_doc_agg.set_xticklabels(tick_labels_doc, rotation=0, ha="center", fontsize=9.0) # Tamaño de fuente aumentado
            if n_total_respuestas_docente_agg > 0: add_styled_paragraph(doc, f'Total de respuestas consolidadas para el docente: {n_total_respuestas_docente_agg}', font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=6)
            plt.subplots_adjust(top=0.88, bottom=0.25, left=0.08, right=0.95)
            filename_doc_agg = f"agregado_{str(docente_agg)[:30].replace('/', '-').replace(' ', '_').replace(',', '')}.png"
            ruta_imagen_doc_agg = os.path.join(carpeta_graficas, filename_doc_agg)
            plt.savefig(ruta_imagen_doc_agg, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_doc_agg)
            try: doc.add_picture(ruta_imagen_doc_agg, width=Inches(6.5))
            except Exception as e: print(f"Error al añadir imagen de docente {docente_agg}: {e}")
            add_styled_paragraph(doc, "", space_after_pt=15)
        else: add_styled_paragraph(doc, "No hay datos suficientes para gráfico agregado de este docente.", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY)
        if docente_agg_idx < len(docentes) -1 : doc.add_page_break()
    doc.add_page_break()

    # --- 4. EVALUACIÓN SYLLABUS POR DOCENTE Y ESPACIO ACADÉMICO (Renumerado y Renombrado) ---
    title4 = "4. EVALUACIÓN SYLLABUS POR DOCENTE Y ESPACIO ACADÉMICO" # Corregido nombre de variable
    add_styled_paragraph(doc, title4, font_name='Arial', font_size=16, is_bold=True, align=WD_ALIGN_PARAGRAPH.CENTER, space_before_pt=12, space_after_pt=12, color_hex='003366', is_heading_for_toc=True, toc_level=0, collect_toc_entry_if_heading=collect_toc_entry_mode, toc_display_text=title4)
    add_styled_paragraph(doc, "", space_after_pt=10)

    for docente_idx, docente_detalle_nombre in enumerate(docentes):
        # Añadir nombre del docente
        add_styled_paragraph(doc, f"Docente: {docente_detalle_nombre}", font_name='Arial', font_size=14, is_bold=True, space_before_pt=10, space_after_pt=8, color_hex='1a73e8', collect_toc_entry_if_heading=False)
        add_styled_paragraph(doc, "", space_after_pt=10)

        espacios_docente = df[df['Docente del Espacio Académico'] == docente_detalle_nombre]['Espacio Académico a evaluar'].dropna().unique()
        if not espacios_docente.any():
            add_styled_paragraph(doc, "No se encontraron espacios académicos para este docente.", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY)
            # Añadir salto de página si NO es el último docente en general
            if docente_idx < len(docentes) - 1:
                doc.add_page_break()
            continue # Saltar al siguiente docente

        for espacio_idx, espacio in enumerate(espacios_docente):
            # Añadir salto de página ANTES del segundo, tercero, etc., espacio del MISMO docente
            if espacio_idx > 0 :
                 doc.add_page_break()

            # Añadir subtítulo del espacio y gráfica
            add_styled_paragraph(doc, f"Espacio Académico: {espacio}", font_name='Arial', font_size=12, is_bold=True, space_before_pt=8, space_after_pt=6, color_hex='202124', collect_toc_entry_if_heading=False)
            add_styled_paragraph(doc, "", space_after_pt=6)
            df_filtrado = df[(df['Docente del Espacio Académico'] == docente_detalle_nombre) & (df['Espacio Académico a evaluar'] == espacio)]
            if df_filtrado.empty:
                add_styled_paragraph(doc, "No hay datos para este espacio académico.", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY)
                continue # Saltar al siguiente espacio

            datos_espacio = []
            n_respuestas_espacio = df_filtrado[preguntas[0]].count() if preguntas and preguntas[0] in df_filtrado.columns else 0
            # ... (lógica de preparación de datos para gráfico de espacio) ...
            for pregunta_texto_completo in preguntas:
                if pregunta_texto_completo not in df_filtrado.columns: continue
                df_filtrado_pregunta_valida = df_filtrado[df_filtrado[pregunta_texto_completo].notna()]
                total_respuestas_pregunta_espacio = len(df_filtrado_pregunta_valida)
                if total_respuestas_pregunta_espacio > 0:
                    conteo_espacio = df_filtrado_pregunta_valida[pregunta_texto_completo].value_counts().reindex(orden_etiquetas, fill_value=0)
                    porcentajes_espacio = (conteo_espacio / total_respuestas_pregunta_espacio) * 100
                    for respuesta, porcentaje in porcentajes_espacio.items():
                        datos_espacio.append({'Pregunta_Grafico': pregunta_texto_completo, 'Respuesta': respuesta, 'Porcentaje': porcentaje, 'Frecuencia': conteo_espacio[respuesta]})
            if datos_espacio:
                # ... (código de generación y guardado del gráfico de espacio) ...
                df_plot_espacio = pd.DataFrame(datos_espacio)
                df_plot_espacio['Pregunta_Grafico'] = pd.Categorical(df_plot_espacio['Pregunta_Grafico'], categories=preguntas, ordered=True)
                df_plot_espacio = df_plot_espacio.sort_values('Pregunta_Grafico')
                fig_esp, ax_esp = plt.subplots(figsize=(11, 7)); fig_esp.set_facecolor('white'); ax_esp.set_facecolor('#f8f9fa')
                sns.barplot(data=df_plot_espacio, x='Pregunta_Grafico', y='Porcentaje', hue='Respuesta', palette=paleta_colores, hue_order=orden_etiquetas, ax=ax_esp, width=0.8, saturation=0.85)
                ax_esp.set_xlabel(''); ax_esp.set_ylabel('Porcentaje (%)', fontsize=10, labelpad=10)
                ax_esp.spines['top'].set_visible(False); ax_esp.spines['right'].set_visible(False); ax_esp.spines['left'].set_linewidth(0.5); ax_esp.spines['bottom'].set_linewidth(0.5)
                ax_esp.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7, color='#dddddd'); ax_esp.set_yticks(range(0, 101, 20)); ax_esp.set_yticklabels([f'{x}%' for x in range(0, 101, 20)], fontsize=9)
                handles, labels = ax_esp.get_legend_handles_labels()
                if ax_esp.get_legend() is not None: ax_esp.get_legend().remove()
                legend_esp = fig_esp.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.97), ncol=len(orden_etiquetas), title='Respuesta', fontsize=9.5, frameon=False)
                plt.setp(legend_esp.get_title(), fontweight='bold', fontsize=10.5)
                tick_labels_esp = [textwrap.fill(pregunta, width=20, break_long_words=False, replace_whitespace=False) for pregunta in preguntas]
                ax_esp.set_xticks(range(len(preguntas)))
                ax_esp.set_xticklabels(tick_labels_esp, rotation=0, ha="center", fontsize=9.0) # Tamaño de fuente aumentado
                add_styled_paragraph(doc, f"Total de respuestas para este espacio: {n_respuestas_espacio}", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY, space_after_pt=6)
                plt.subplots_adjust(top=0.88, bottom=0.25, left=0.08, right=0.95)
                filename_espacio = f"{str(docente_detalle_nombre)[:20]}_{str(espacio)[:20]}".replace('/', '-').replace(' ', '_').replace(',', '') + ".png"
                ruta_imagen_espacio = os.path.join(carpeta_graficas, filename_espacio)
                plt.savefig(ruta_imagen_espacio, dpi=300, bbox_inches='tight', format='png'); plt.close(fig_esp)
                try: doc.add_picture(ruta_imagen_espacio, width=Inches(6.5))
                except Exception as e: print(f"Error al añadir imagen de espacio {espacio}: {e}")
                add_styled_paragraph(doc, "", space_after_pt=15)
            else: add_styled_paragraph(doc, "No hay datos suficientes para gráfico de este espacio.", font_name='Arial', font_size=10, align=WD_ALIGN_PARAGRAPH.JUSTIFY)

        # Añadir page break DESPUÉS de procesar todos los espacios de un docente, si NO es el último docente
        if docente_idx < len(docentes) - 1:
            doc.add_page_break()


# --- FASE 1: Generar contenido y recolectar información para la TOC ---
temp_doc_for_content_generation = Document()
print("FASE 1: Iniciando generación de contenido y recolección de TOC...")
toc_entries = []
_global_bookmark_id_counter = 0
build_introductory_section(temp_doc_for_content_generation, total_respuestas_global, collect_toc_entry_mode=True)
build_main_content_graphics(temp_doc_for_content_generation, collect_toc_entry_mode=True)
print(f"FASE 1: Finalizada. Entradas de TOC recolectadas: {len(toc_entries)}")


# --- FASE 2: Ensamblar el documento final ---
try:
    if os.path.exists(ruta_plantilla_docx):
        final_document = Document(ruta_plantilla_docx)
        print(f"FASE 2: Documento final creado usando plantilla: {ruta_plantilla_docx}")
    else:
        final_document = Document()
        print(f"FASE 2: Documento final creado sin plantilla.")
except Exception as e:
    print(f"FASE 2: Error al cargar plantilla: {e}. Creando documento vacío.")
    final_document = Document()

# 1. Portada
print("FASE 2: Añadiendo Portada...")
build_portada(final_document)
final_document.add_page_break()

# 2. Tabla de Contenido
print("FASE 2: Añadiendo Tabla de Contenido...")
build_table_of_contents(final_document, toc_entries)
final_document.add_page_break()

# 3. Contenido Principal (Intro + Gráficas)
print("FASE 2: Recreando contenido principal en el documento final...")
_global_bookmark_id_counter = 0 # Resetear contador de IDs para la FASE 2 (CRUCIAL)
build_introductory_section(final_document, total_respuestas_global, collect_toc_entry_mode=False)
final_document.add_page_break() # Salto de página después de la intro
build_main_content_graphics(final_document, collect_toc_entry_mode=False)
print("FASE 2: Ensamblaje del documento final completado.")


# --- GUARDAR EL DOCUMENTO DOCX Y CONVERTIR A PDF ---
try:
    final_document.save(temp_docx_output_path)
    print(f"Informe DOCX (v4.10) guardado en: {temp_docx_output_path}")

    print(f"Intentando convertir '{temp_docx_output_path}' a PDF usando LibreOffice...")
    # ... (resto del código de conversión a PDF sin cambios) ...
    output_dir_for_pdf = os.path.dirname(final_pdf_output_path)
    if not output_dir_for_pdf: output_dir_for_pdf = "."
    else: os.makedirs(output_dir_for_pdf, exist_ok=True)
    libreoffice_executables = ['soffice', 'libreoffice']
    process_result = None; executable_found = False
    for executable in libreoffice_executables:
        command = [executable, '--headless', '--convert-to', 'pdf', '--outdir', output_dir_for_pdf, temp_docx_output_path]
        try:
            print(f"Intentando con el comando: {' '.join(command)}")
            process_result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=False, timeout=180)
            if process_result.returncode == 0:
                print(f"Comando '{executable}' ejecutado exitosamente."); executable_found = True; break
            else:
                print(f"Comando '{executable}' falló (código {process_result.returncode}): {process_result.stderr.decode('utf-8', errors='ignore')}")
        except FileNotFoundError:
            print(f"Error: El ejecutable '{executable}' no fue encontrado en el PATH.")
        except subprocess.TimeoutExpired:
            print(f"Error: El comando '{executable}' excedió el tiempo límite de 180 segundos.")
        except Exception as e:
            print(f"Error inesperado ejecutando '{executable}': {e}")
    pdf_gen_path = os.path.join(output_dir_for_pdf, os.path.splitext(os.path.basename(temp_docx_output_path))[0] + ".pdf")
    if executable_found and process_result and process_result.returncode == 0 and os.path.exists(pdf_gen_path):
        if pdf_gen_path != final_pdf_output_path:
            try:
                os.rename(pdf_gen_path, final_pdf_output_path)
                print(f"Informe PDF final (v4.10) creado en: {final_pdf_output_path}")
            except OSError as e:
                print(f"Error al renombrar PDF: {e}. PDF generado en: {pdf_gen_path}")
        else:
            print(f"Informe PDF final (v4.10) creado en: {final_pdf_output_path}")
    elif executable_found:
        print("-" * 50 + "\nERROR: Conversión DOCX a PDF falló aunque se encontró LibreOffice." + "-" * 50)
        if process_result: print(f"Último código de retorno: {process_result.returncode}\nÚltimo stderr: {process_result.stderr.decode('utf-8', errors='ignore')}")
        if not os.path.exists(pdf_gen_path): print(f"Archivo PDF esperado ({pdf_gen_path}) no encontrado.")
        print(f"El archivo DOCX generado está en: {temp_docx_output_path}")
    else:
        print("-" * 50 + "\nADVERTENCIA: LibreOffice no encontrado." + "-" * 50)
        print("La conversión a PDF no se pudo realizar. DOCX está en: {temp_docx_output_path}")
except Exception as e:
    print(f"Error general guardando/convirtiendo: {e}"); import traceback; traceback.print_exc()
print("Proceso v4.10 completado.")


Advertencia: Locale 'es_CO.UTF-8' o 'es_ES.UTF-8' no encontrado. Usando locale por defecto.
Entorno de Google Colab detectado (solo informativo).
Total de respuestas global calculado: 821
FASE 1: Iniciando generación de contenido y recolección de TOC...
FASE 1: Finalizada. Entradas de TOC recolectadas: 13
FASE 2: Documento final creado usando plantilla: /content/drive/MyDrive/EVALUACIÓN DOCENTE/DATOS-PLANTILLA/plantilla.docx
FASE 2: Añadiendo Portada...
FASE 2: Añadiendo Tabla de Contenido...
FASE 2: Recreando contenido principal en el documento final...
FASE 2: Ensamblaje del documento final completado.
Informe DOCX (v4.10) guardado en: /content/drive/MyDrive/EVALUACIÓN DOCENTE/INFORME/Informe_Evaluacion_Syllabus_v4_10.docx
Intentando convertir '/content/drive/MyDrive/EVALUACIÓN DOCENTE/INFORME/Informe_Evaluacion_Syllabus_v4_10.docx' a PDF usando LibreOffice...
Intentando con el comando: soffice --headless --convert-to pdf --outdir /content/drive/MyDrive/EVALUACIÓN DOCENTE/INFORME /co